**Importing required libraries.**

In [7]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
np.random.seed(500)
Corpus = pd.read_csv('bbc-text.csv',delimiter=',',encoding='latin-1')
Corpus.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [9]:
Corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [10]:
# 1. Removing Blank Spaces
Corpus['text'].dropna(inplace=True)
# 2. Changing all text to lowercase
Corpus['text_original'] = Corpus['text']
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# 3. Tokenization-In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# 4. Remove Stop words, Non-Numeric and perfoming Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

Corpus.head()

,category,text,text_original
0,tech,"[tv, future, in, the, hands, of, viewers, with...",tv future in the hands of viewers with home th...
1,business,"[worldcom, boss, left, books, alone, former, w...",worldcom boss left books alone former worldc...
2,sport,"[tigers, wary, of, farrell, gamble, leicester,...",tigers wary of farrell gamble leicester say ...
3,sport,"[yeading, face, newcastle, in, fa, cup, premie...",yeading face newcastle in fa cup premiership s...
4,entertainment,"[ocean, s, twelve, raids, box, office, ocean, ...",ocean s twelve raids box office ocean s twelve...


In [11]:
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [25]:
Corpus.drop(['text'], axis=1)
output_path = 'preprocessed_data.csv'
Corpus.to_csv(output_path, index=False)
Corpus.head()

,category,text,text_original,text_final
0,tech,"[tv, future, in, the, hands, of, viewers, with...",tv future in the hands of viewers with home th...,"['tv', 'future', 'hand', 'viewer', 'home', 'th..."
1,business,"[worldcom, boss, left, books, alone, former, w...",worldcom boss left books alone former worldc...,"['worldcom', 'bos', 'leave', 'book', 'alone', ..."
2,sport,"[tigers, wary, of, farrell, gamble, leicester,...",tigers wary of farrell gamble leicester say ...,"['tiger', 'wary', 'farrell', 'gamble', 'leices..."
3,sport,"[yeading, face, newcastle, in, fa, cup, premie...",yeading face newcastle in fa cup premiership s...,"['yeading', 'face', 'newcastle', 'fa', 'cup', ..."
4,entertainment,"[ocean, s, twelve, raids, box, office, ocean, ...",ocean s twelve raids box office ocean s twelve...,"['ocean', 'twelve', 'raid', 'box', 'office', '..."


<a href="preprocessed_data.csv"> Output file</a>

In [32]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['category'],test_size=0.3)
print(Corpus['text_final'].shape)

(2225,)


In [14]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)


In [21]:
print(Train_X_Tfidf.shape)


(1557, 5000)


In [17]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)






Naive Bayes Accuracy Score ->  97.30538922155688


In [18]:
Train_X_Tfidf.shape

(1557, 5000)

In [19]:


print(classification_report(Test_Y, predictions_NB))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       161
           1       0.99      0.98      0.99       120
           2       0.94      0.97      0.95       125
           3       0.99      0.99      0.99       136
           4       0.98      0.96      0.97       126

    accuracy                           0.97       668
   macro avg       0.97      0.97      0.97       668
weighted avg       0.97      0.97      0.97       668



In [20]:
from sklearn.ensemble import RandomForestClassifier

from yellowbrick.classifier import ClassPredictionError

# Instantiate the classification model and visualizer
visualizer = ClassPredictionError(
    Naive, classes=Encoder.classes_
)

# Fit the training data to the visualizer
visualizer.fit(Train_X_Tfidf,Train_Y)

# Evaluate the model on the test data
visualizer.score(Test_X_Tfidf, Test_Y)

# Draw visualization
g = visualizer.poof()


ModuleNotFoundError: No module named 'yellowbrick'

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  98.35329341317365


In [ ]:
print(classification_report(Test_Y,predictions_SVM))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       161
           1       0.98      1.00      0.99       120
           2       0.98      0.98      0.98       125
           3       0.99      0.99      0.99       136
           4       0.99      0.99      0.99       126

    accuracy                           0.98       668
   macro avg       0.98      0.98      0.98       668
weighted avg       0.98      0.98      0.98       668

